In [1]:
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from keras.applications import resnet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from sklearn.utils import shuffle
import numpy.random as rng
import os
import re
from PIL import Image
import pickle

import glob

c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\rahul\appd

In [2]:
anchor=Input((120,120,3))
positive=Input((120,120,3))
negetive=Input((120,120,3))
convnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(120,120,3))

c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
for layer in convnet.layers[:-1]:
    layer.trainable=False

In [4]:
class TripletLossLayer(Layer):
    def  __init__(self,alpha,**kwargs):
        self.alpha=alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        p_dist = K.sum(K.square(anchor-positive), axis=-1)
        n_dist = K.sum(K.square(anchor-negative), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

In [5]:
x = convnet.output
x = Flatten()(x)
x = Dense(1024,kernel_regularizer=l2(2e-3))(x)
convnet = Model(inputs=convnet.input, outputs=x)


In [6]:
encoded_a = convnet(anchor)
encoded_p = convnet(positive)
encoded_n = convnet(negetive)

Euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

loss_layer=TripletLossLayer(alpha=0.2,name="triplet_loss_layer")([encoded_a,encoded_p,encoded_n])

In [7]:
model = Model(inputs=[anchor,positive,negetive],outputs=loss_layer)

In [8]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         57143168    input_1[0][0]                    
                                                                 input_2[0][0]              

In [20]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.00006)
model.compile(loss=None,optimizer=optimizer)


c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output triplet_loss_layer missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to triplet_loss_layer.
  'be expecting any data to be passed to {0}.'.format(name))


In [9]:
X=[]
y=[]
curr_y=0
n_dict={}
root='./PINS'
for name in os.listdir(root):
    n_dict[name.split("_")[1]]=curr_y
    curr_y+=1

In [10]:
root='./PINS'
root
for name in os.listdir(root):
    namepath=os.path.join(root,name)
    p_name=name.split("_")[1]
    print(p_name)
    for image in os.listdir(namepath):
        image_path=os.path.join(namepath,image)
        new_image=Image.open(image_path)
        size=(120,120)
        new_image=new_image.resize(size,Image.ANTIALIAS)
        final_image=np.array(new_image,'uint8')/255.0
        ##print(final_image.shape)
        shape=final_image.shape
        final_image=final_image.reshape(shape[0],shape[1],3)
        X.append(np.stack(final_image))
        y.append(n_dict[p_name])
        

Aaron Paul
alexandra daddario
Alvaro Morte
alycia debnam carey face
Amanda Crew
Amaury Nolasco
amber heard face
Anna Gunn
anne hathaway
barbara palvin face
bellamy blake face
Benedict Cumberbatch
Betsy Brandt
bill gates
Brenton Thwaites
brie larson
Brit Marling
Bryan Cranston
Caity Lotz
Cameron Monaghan
chadwick boseman face
Chance Perdomo
Chris Evans
Chris Pratt
Cobie Smulders
Danielle Panabaker
Dave Franco
david mazouz
Dominic Purcell
drake
dua lipa face
Dwayne Johnson
eliza taylor
elizabeth olsen face
elon musk
Emilia Clarke
Emily Bett Rickards
Emma Stone
emma watson face
gal gadot face
grant gustin face
Gwyneth Paltrow
Henry Cavil
jason isaacs
Jason Momoa
jeff bezos
Jeremy Renner
Jesse Eisenberg
Jim Parsons
Jon Bernthal
Josh Radnor
kiernan shipka
Kit Harington
kristen stewart face
Krysten Ritter
Kumail Nanjiani
lindsey morgan face
Maisie Williams
margot robbie face
maria pedraza
Mark Ruffalo
mark zuckerberg
Martin Starr
Melissa benoit
miguel herran
Mike Colter
millie bobby brown
Mo

In [11]:
featured_img = convnet.predict(np.ones((1,120,120,3)))
featured_img

array([[-0.32670483,  0.5797118 ,  0.1586476 , ...,  0.26011398,
         0.42816803, -0.03649538]], dtype=float32)

In [12]:
featured_img.shape

(1, 1024)

In [13]:
dataset_train=[]
 
for n in range(100):
    images_class_n=np.asarray([row for idx,row in enumerate(X) if y[idx]==n])
    dataset_train.append(images_class_n/255)
    

In [14]:
dataset_train[0].shape
print((dataset_train[5].shape))

(113, 120, 120, 3)


In [15]:
def get_batch_random(batch_size,s="train"):
    
    if s=="train":
        X=dataset_train
        nb_classes=len(dataset_train)
    else:
        X=dataset_test
        nb_classes=(len(dataset_test))
    m,w,h,c=X[0].shape
    
    triplets=[np.zeros((batch_size,h,w,c)) for i in range(3)]
    
    
    for i in range(batch_size):
        anchor_class=np.random.randint(0,nb_classes)
        nb_sample_available_for_class_AP=X[anchor_class].shape[0]
        
        [idx_A,idx_P]=np.random.choice(nb_sample_available_for_class_AP,size=2,replace=False)
        
        negative_class = (anchor_class + np.random.randint(1,nb_classes)) % nb_classes
        nb_sample_available_for_class_N = X[negative_class].shape[0]
        
        idx_N = np.random.randint(0, nb_sample_available_for_class_N)
        
        triplets[0][i,:,:,:] = X[anchor_class][idx_A,:,:,:]
        triplets[1][i,:,:,:] = X[anchor_class][idx_P,:,:,:]
        triplets[2][i,:,:,:] = X[negative_class][idx_N,:,:,:]
        
    return triplets

def compute_dist(a,b):
    return np.sum(np.square(a-b))

def get_batch_hard(draw_batch_size,hard_batchs_size,norm_batchs_size,network,s="train"):
    
    if s == 'train':
        X = dataset_train
    else:
        X = dataset_test

    m, w, h,c = X[0].shape  
    
    studybatch = get_batch_random(draw_batch_size,s)
        
    studybatchloss = np.zeros((draw_batch_size))
    
    A = network.predict(studybatch[0])
    P = network.predict(studybatch[1])
    N = network.predict(studybatch[2])

    studybatchloss = np.sum(np.square(A-P),axis=1) - np.sum(np.square(A-N),axis=1)
    
    selection = np.argsort(studybatchloss)[::-1][:hard_batchs_size]
    
    selection2 = np.random.choice(np.delete(np.arange(draw_batch_size),selection),norm_batchs_size,replace=False)
    
    selection = np.append(selection,selection2)
    
    triplets = [studybatch[0][selection,:,:,:], studybatch[1][selection,:,:,:], studybatch[2][selection,:,:,:]]
    
    return triplets
    
    
        

In [16]:
triplets=get_batch_random(2)
print(len(triplets))
print("shapes i batch A: {0} P:{1} N :{2}".format(triplets[0].shape,triplets[1].shape,triplets[2].shape))

hardtriplets=get_batch_hard(50,1,1,convnet)
print("Shapes in the hardbatch A:{0} P:{1} N:{2}".format(hardtriplets[0].shape, hardtriplets[1].shape, hardtriplets[2].shape))

3
shapes i batch A: (2, 120, 120, 3) P:(2, 120, 120, 3) N :(2, 120, 120, 3)
Shapes in the hardbatch A:(2, 120, 120, 3) P:(2, 120, 120, 3) N:(2, 120, 120, 3)


In [22]:
evaluate_every = 200  # interval for evaluating on one-shot tasks
batch_size = 32
n_iteration = 15000        # No. of training iterations
n_val = 200           # how many one-shot tasks to validate on


In [18]:
import time

In [ ]:
print("starting traning process!")
print("--------------------------")
t_start=time.time()
dummy_target=[np.zeros((batch_size,15)) for i in range(3)]
for i in range(1,n_iter+1):
    triplets=get_batch_hard(200,16,16,convnet)
    loss=model.train_on_batch(triplets,None)
    n_iteration+=1
    if i% evaluate_every==0:
        print("\n ------------- \n")
        print("[{3}] Time for {0} iterations: {1:.1f} mins, Train Loss: {2}".format(i, (time.time()-t_start)/60.0,loss,n_iteration))
        
        

starting traning process!
--------------------------

 ------------- 

[15400] Time for 200 iterations: 8.5 mins, Train Loss: 10169.38671875
